In [1]:
from pyce import raster as pyraster 
from pyce import shape as pyshape
import pyce

In [2]:
import ee, geemap, eemont
import rioxarray
import geopandas as gpd
from geopandas import GeoSeries as Gs
import pandas as pd
import os

import folium

In [11]:
ee.Authenticate()
ee.Initialize()

-----------------------------------------

ts and geometry import

In [3]:
# Dossier contenant les fichiers raster
folder = "C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1"

# Charger tous les rasters de 1984 à 2024
glacier_raster = {
    year: rioxarray.open_rasterio(f"{folder}/GLACIER_{year}.tif")
    for year in range(1984, 2025) 
}

In [4]:
# Dossier contenant les fichiers raster
folder = "C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1"

# Charger tous les rasters de 1984 à 2024
water_rasters = {
    year: rioxarray.open_rasterio(f"{folder}/WATER_{year}.tif")
    for year in range(1984, 2025)
}


In [5]:
water_raster_2014_slope_10=rioxarray.open_rasterio("C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1/WATER_2014_slope_10.tif")

In [6]:
geom_RGI_FR_2015_gdf=gpd.read_file('C:/Users/ronir/Desktop/Stage_Carrtel_2025/QGIS/Export_RGI_LIA/rgi_fr_2015.kml')
geom_RGI_FR_2000_gdf=gpd.read_file('C:/Users/ronir/Desktop/Stage_Carrtel_2025/QGIS/Export_RGI_LIA/rgi_fr_2000.kml')

In [7]:
geom_RGI_FR_2015_to_export=pyshape.remove_linestring(geom_RGI_FR_2015_gdf)
geom_RGI_FR_2000_to_export=pyshape.remove_linestring(geom_RGI_FR_2000_gdf)

In [10]:
gdf_lia_vdef=gpd.read_file("C:/Users/ronir/Desktop/Stage_Carrtel_2025/QGIS/Shape LIA/transfer_9869808_files_c7ca226a/Glaciers_1850_final_mourey2025.shp")

importing lake inventory

In [11]:
lake_shape=gpd.read_file('C:/Users/ronir/Desktop/Stage_Carrtel_2025/Inventaire_Lac_alpin/Inventaire_lacs_2022/Inventaire_lacs_2022/inventaire_tous_lacs_2022.shp')  
lake_shape = lake_shape.to_crs(gdf_lia.crs)

In [12]:
lake_shape_clipped=gpd.read_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1/Shapefile/lake_shape_clipped.shp')

In [13]:
lake_classif_inrae_clean_gt900m2 = gpd.read_file("C:/Users/ronir/Desktop/Stage_Carrtel_2025/Resultats/validation_lake_classif/lake_classif_inrae_clean_gt900m2/lake_classif_inrae_clean_gt900m2.dbf")

-----------------------------------------

# FUNCTIONS 

In [14]:
def add_year_column(gdf, year):
    """
    Ajoute une colonne 'année' à un GeoDataFrame, remplie avec une année spécifique.
    
    Args:
        gdf (geopandas.GeoDataFrame): Le GeoDataFrame d'entrée.
        year (int): L'année à ajouter dans la nouvelle colonne 'année'.
        
    Returns:
        geopandas.GeoDataFrame: Le GeoDataFrame avec la colonne 'année' ajoutée.
    """
    gdf['year'] = year
    return gdf

In [15]:
def multi_raster_to_gdf(rasters,tresh_value):
    """
    Polygonise plusieurs couches raster (par année) et retourne un GeoDataFrame concaténé.
    
    Paramètres:
        rasters (xarray.Dataset): Dataset avec une variable par année (nommée par année ou identifiant temporel).
    
    Retour:
        GeoDataFrame: géométries polygonisées concaténées avec colonne 'year'.
    """
    gdf_list = []

    for year_str, raster in rasters.items():
        # Convertir l'identifiant en année (assume que c’est un str représentant l’année, sinon adapter)
        year = int(str(year_str))

        # Polygonisation
        shape = pyraster.polygonize_raster(
            raster,
            mask=(raster == tresh_value),
            transform=raster.rio.transform()
        )

        # GeoDataFrame + ajout de la colonne année
        gdf = gpd.GeoDataFrame(
            geometry=[shape],
            crs=raster.rio.crs
        ).explode(ignore_index=True)
        gdf["year"] = year

        gdf_list.append(gdf)

    # Concaténer tous les GeoDataFrames
    gdf_all = pd.concat(gdf_list, ignore_index=True)

    return gdf_all, gdf_list

In [16]:
def filter_water_by_glacier_gpd(gdf_water_dict, gdf_glacier_dict):
    """
    Filtre les géométries des eaux à l'intérieur et à l'extérieur des glaciers sur plusieurs années,
    et retourne deux GeoDataFrames concaténés avec une colonne 'year'.

    Args:
        gdf_water_dict (dict[int, geopandas.GeoDataFrame]): GeoDataFrames d'eau par année.
        gdf_glacier_dict (dict[int, geopandas.GeoDataFrame]): GeoDataFrames de glaciers par année.

    Returns:
        tuple:
            - GeoDataFrame des eaux dans les glaciers (avec colonne 'year')
            - GeoDataFrame des eaux hors glaciers (avec colonne 'year')
    """
    water_in_list = []
    water_out_list = []

    for year in range(1984, 2025):
        
        gdf_water = gdf_water_dict[gdf_water_dict["year"]==year]
        gdf_glacier = gdf_glacier_dict[gdf_glacier_dict["year"]==year]

        # Calculer les géométries à l'intérieur des glaciers
        gdf_in = gdf_water.sjoin(gdf_glacier, how='inner', predicate='within')

        #Calculer celles en dehors des glaciers
        gdf_out = gdf_water.loc[~gdf_water.index.isin(gdf_in.index)]

        water_in_list.append(gdf_in)
        water_out_list.append(gdf_out)

    # Concaténer tous les résultats en deux GeoDataFrames
    water_in_glacier = pd.concat(water_in_list, ignore_index=True)
    water_outside = pd.concat(water_out_list, ignore_index=True)

    return water_in_glacier, water_outside

In [17]:
def exclude_water_inside_geometry(water_gdf,geom):
    """
    Exclut les polygones de water_gdf entièrement contenus dans geometry.
    """
    water_in_list = []
    water_out_list = []
       
    water_in_geom = water_gdf.sjoin(geom, how='inner', predicate='within')
    water_outside_geom = water_gdf.loc[~water_gdf.index.isin(water_in_geom.index)]

    water_in_list.append(water_in_geom)
    water_out_list.append(water_outside_geom )

    water_in_geom_pd = pd.concat(water_in_list, ignore_index=True)
    water_outside_geom_pd = pd.concat(water_out_list, ignore_index=True)

    return  water_outside_geom_pd
    

In [18]:
def exclude_inside_RGI_water(gdf, rgi_gdf,year_rgi):
    """
    Exclut les polygones de water_gdf entièrement contenus dans rgi_gdf
    uniquement si leur année est inférieure ou égale à celle du RGI.
    
    Args:
        water_gdf (GeoDataFrame): GeoDataFrame des plans d'eau avec une colonne 'année'.
        rgi_gdf (GeoDataFrame): GeoDataFrame des glaciers avec une colonne 'année'.

    Returns:
        GeoDataFrame: GeoDataFrame des plans d'eau filtré.
    """
    water_in_rgi_list=[]
    water_outside_RGI_list=[]
    
    for year in range(1984, year_rgi+1):

        gdf_to_filter = gdf[gdf["year"]==year]
        
        # Jointure spatiale
        gdf_in = gdf_to_filter.sjoin(rgi_gdf, how='inner', predicate='within')
        gdf_out = gdf_to_filter.loc[~gdf_to_filter.index.isin(gdf_in.index)]

        water_outside_RGI_list.append(gdf_out)
        water_outside_RGI = pd.concat(water_outside_RGI_list, ignore_index=True)
        
    return  water_outside_RGI
    

In [19]:
def exclude_outside_RGI_glacier(gdf, rgi_gdf,year_rgi):
    """
    Exclut les polygones de glacier_gdf entièrement non contenus dans rgi_gdf
    uniquement si leur année est inférieure ou égale à celle du RGI.
    
    Args:
        glacier_gdf (GeoDataFrame): GeoDataFrame des plans d'eau avec une colonne 'année'.
        rgi_gdf (GeoDataFrame): GeoDataFrame des glaciers avec une colonne 'année'.

    Returns:
        GeoDataFrame: GeoDataFrame des plans d'eau filtré.
    """
    glacier_inside_rgi_list=[]
    glacier_outside_rgi_list=[]
    
    for year in range(1984, year_rgi+1):

        gdf_to_filter = gdf[gdf["year"]==year]
        
        # Jointure spatiale
        gdf_in = gdf_to_filter.sjoin(rgi_gdf, how='inner', predicate='intersects')
        gdf_to_conserve = gdf_to_filter.loc[gdf_to_filter.index.isin(gdf_in.index)]

        glacier_inside_rgi_list.append(gdf_to_conserve)
        glacier_inside_rgi = pd.concat(glacier_inside_rgi_list, ignore_index=True)
        
    return  glacier_inside_rgi
    

In [20]:
# Filtering with 5 years
def water_time_filtering_propre(water_gdf):
    """
    Filtre les polygones d'eau pour ne conserver que ceux qui s'intersectent 
    sur plusieurs années **futures** consécutives (croissant), de 1984 à 2024.

    - 1984 à 2020 : intersection sur 5 années (année, +1, +2, +3, +4)
    - 2021 : intersection sur 4 années (année, +1, +2, +3)
    - 2022 : intersection sur 3 années (année, +1, +2)
    - 2023 : intersection sur 2 années (année, +1)
    - 2024 : utilisée seule (aucun test)
    """
    start_year=1984
    end_year=2024
    years = range(start_year, end_year+1)
    gdf_time_filtered_list = []

    for year in years:
        # Nombre d'années à considérer
        dy = min(5, end_year - year + 1)
       
        gdf_base = water_gdf[water_gdf['year'] == year]
        gdf_future = water_gdf[(water_gdf['year'] > year) & (water_gdf['year'] <= year + dy)]

        if dy == 1:
            # Cas 1984, on garde tout
            filtered = gdf_base
            gdf_time_filtered_list.append(filtered)
        else: 
            joined = gdf_base.sjoin(gdf_future, how='inner', predicate='intersects')
            joined['index_left'] = joined.index
            # Années réellement intersectées dans cette plage
            expected_years = set(gdf_future['year'].unique())
            
            # Pour chaque géométrie de gdf_base, collecter les années qu'elle intersecte
            years_by_geom = joined.groupby('index_left')['year_right'].apply(set)

            # Garder seulement les géométries qui intersectent toutes les années attendues
            valid_indices = years_by_geom[years_by_geom == expected_years].index

            # Filtrage
            filtered = gdf_base.loc[gdf_base.index.isin(valid_indices)]
            gdf_time_filtered_list.append(filtered)

    gdf_time_filtered = pd.concat(gdf_time_filtered_list, ignore_index=True)
    return gdf_time_filtered

In [21]:
# Each polygone has to be detected 80% of the years since its first detection
# Efficace à l'échelle des alpes sans besoin de clip au lac, pollution restante en haut des domes, 

def water_time_filtering_propre_total(water_gdf):
    """
    Filtre les polygones d'eau pour ne conserver que ceux qui s'intersectent 
    sur plusieurs années **futures** consécutives (croissant), de 1984 à 2024.

    - 1984 à 2020 : intersection sur 5 années (année, +1, +2, +3, +4)
    - 2021 : intersection sur 4 années (année, +1, +2, +3)
    - 2022 : intersection sur 3 années (année, +1, +2)
    - 2023 : intersection sur 2 années (année, +1)
    - 2024 : utilisée seule (aucun test)
    """
    end_year=2024
    start_year=1984
    years = range(start_year, end_year+1)
    gdf_time_filtered_list = []

    for year in years:
        # Nombre d'années à considérer
        dy = end_year - year + 1
       
        gdf_base = water_gdf[water_gdf['year'] == year]
        gdf_future = water_gdf[(water_gdf['year'] > year) & (water_gdf['year'] <= year + dy)]

        if dy == 1:
            # Cas 1984, on garde tout
            filtered = gdf_base
            gdf_time_filtered_list.append(filtered)
        else: 
            joined = gdf_base.sjoin(gdf_future, how='inner', predicate='intersects')
            joined['index_left'] = joined.index
            # Années réellement intersectées dans cette plage
            expected_years = set(gdf_future['year'].unique())

            # Pour chaque géométrie de gdf_base, collecter les années qu'elle intersecte
            years_by_geom = joined.groupby('index_left')['year_right'].apply(set)

            # Garder seulement les géométries qui intersectent toutes les années attendues
            valid_indices = years_by_geom[years_by_geom.apply(lambda years: len(years) >= 0.8 * len(expected_years))].index

            # Filtrage
            filtered = gdf_base.loc[gdf_base.index.isin(valid_indices)]
            gdf_time_filtered_list.append(filtered)

    gdf_time_filtered = pd.concat(gdf_time_filtered_list, ignore_index=True)
    return gdf_time_filtered

In [22]:
def glacier_time_filtering_propre(glacier_gdf):
    """
    Filtre les polygones de glacier pour ne conserver que ceux qui s'intersectent 
    sur toutes les années passées consécutives (décroissant), de 2024 à 1984.

    - 2024 à 1988 : intersection sur 5 années (année, -1, -2, -3, -4)
    - 1987 : intersection sur 4 années (année, -1, -2, -3)
    - 1986 : intersection sur 3 années (année, -1, -2)
    - 1985 : intersection sur 2 années (année, -1)
    - 1984 : utilisée seule (aucun test)
    """
    start_year = 1985
    end_year = 2024
    gdf_time_filtered_list = []

    for year in range(end_year, start_year - 1, -1):
        dy = min(5, year - start_year + 1)

        gdf_base = glacier_gdf[glacier_gdf['year'] == year]
        gdf_past = glacier_gdf[(glacier_gdf['year'] < year) & (glacier_gdf['year'] >= year - dy)]

        if dy == 1:
            gdf_time_filtered_list.append(gdf_base)
        else:
            # Jointure spatiale
            joined = gdf_base.sjoin(gdf_past, how='inner', predicate='intersects')
            
            joined['index_left'] = joined.index
            
            # Années réellement intersectées dans cette plage
            expected_years = set(gdf_past['year'].unique())

            # Pour chaque géométrie de gdf_base, collecter les années qu'elle intersecte
            years_by_geom = joined.groupby('index_left')['year_right'].apply(set)

            # Garder seulement les géométries qui intersectent toutes les années attendues
            valid_indices = years_by_geom[years_by_geom == expected_years].index

            # Filtrage
            filtered = gdf_base.loc[gdf_base.index.isin(valid_indices)]
            gdf_time_filtered_list.append(filtered)

    gdf_time_filtered = pd.concat(gdf_time_filtered_list, ignore_index=True)
    return gdf_time_filtered.sort_values(by='year', ascending=True).reset_index(drop=True)

In [23]:
def glacier_time_filtering_propre_total(glacier_gdf):
    """
    Filtre les polygones de glacier pour ne conserver que ceux qui s'intersectent 
    sur toutes les années passées consécutives (décroissant), de 2024 à 1984.

    - 2024 à 1988 : intersection sur 5 années (année, -1, -2, -3, -4)
    - 1987 : intersection sur 4 années (année, -1, -2, -3)
    - 1986 : intersection sur 3 années (année, -1, -2)
    - 1985 : intersection sur 2 années (année, -1)
    - 1984 : utilisée seule (aucun test)
    """
    start_year = 1985
    end_year = 2024
    gdf_time_filtered_list = []

    for year in range(end_year, start_year - 1, -1):
        dy =  year - start_year + 1

        gdf_base = glacier_gdf[glacier_gdf['year'] == year]
        gdf_past = glacier_gdf[(glacier_gdf['year'] < year) & (glacier_gdf['year'] >= year - dy)]

        if dy == 1:
            gdf_time_filtered_list.append(gdf_base)
        else:
            # Jointure spatiale
            joined = gdf_base.sjoin(gdf_past, how='inner', predicate='intersects')
            joined['index_left'] = joined.index
            # Années réellement intersectées dans cette plage
            expected_years = set(gdf_past['year'].unique())

            # Pour chaque géométrie de gdf_base, collecter les années qu'elle intersecte
            years_by_geom = joined.groupby('index_left')['year_right'].apply(set)

            # Garder seulement les géométries qui intersectent toutes les années attendues
            valid_indices = years_by_geom[years_by_geom.apply(lambda years: len(years) >= 0.9 * len(expected_years))].index

            # Filtrage
            filtered = gdf_base.loc[gdf_base.index.isin(valid_indices)]
            gdf_time_filtered_list.append(filtered)

    gdf_time_filtered = pd.concat(gdf_time_filtered_list, ignore_index=True)
    return gdf_time_filtered.sort_values(by='year', ascending=True).reset_index(drop=True)

In [24]:
# def homogenize_lakes_2015_2016(gdf, year_col='year'):
#     """
#     Homogénéise les géométries des lacs entre 2015 et 2016 :
#     Pour chaque polygone de 2016, ajoute les parties perdues des polygones de 2015 avec lesquels il intersecte.

#     :param gdf: GeoDataFrame avec les colonnes 'year' et 'geometry'.
#     :param year_col: Colonne contenant l'année.
#     :return: GeoDataFrame pour 2016, avec géométries homogénéisées.
#     """
    
#     # Filtrer les années 2015 et 2016
#     gdf = gdf[gdf[year_col].isin([2015, 2016])].copy()

#     gdf_2015 = gdf[gdf[year_col] == 2015].copy()
#     gdf_2016 = gdf[gdf[year_col] == 2016].copy()

#     # Préparer index pour suivi
#     gdf_2016 = gdf_2016.reset_index(drop=True)
#     gdf_2016['index_2016'] = gdf_2016.index

#     # Jointure spatiale : quels polygones de 2015 intersectent ceux de 2016
#     joined = gdf_2015.sjoin(gdf_2016, how='inner', predicate='intersects')

#     # Regrouper par chaque polygone de 2016
#     grouped = joined.groupby('index_2016')
    
#     # Homogénéiser géométries
#     for idx, group in grouped:
#         geom_2016 = gdf_2016.loc[idx, 'geometry']
#         gdf_2015.union(
#         new_geom = geom_2016.union(diff_union)
#         gdf_2016.at[idx, 'geometry'] = new_geom

#     # Nettoyage et retour
#     return gdf_2016.drop(columns='index_2016')

In [25]:
# import geopandas as gpd
# import pandas as pd


# def homogenize_lakes_total(gdf):
#     years = sorted(gdf['year'].unique())

#     # 1) On démarre la liste avec la toute première année 
#     gdf_homogenized_list = [gdf[gdf['year'] == years[0]]]

#     # 2) On boucle sur toutes les années 
#     for year in years:
#         y_n  = year
#         y_n1 = year + 1

#         # 3) On prend les données brutes de chaque année
#         gdf_n  = gdf[gdf['year'] == y_n].copy()
#         gdf_n1 = gdf[gdf['year'] == y_n1].copy()

#         # 4) Préparer index pour le suivi dans gdf_n1
#         gdf_n1 = gdf_n1.reset_index(drop=True)
#         gdf_n1['index_n1'] = gdf_n1.index

#         # 5) Jointure spatiale : qui de N intersecte N+1 ?
#         joined = gdf_n.sjoin(gdf_n1, how='inner', predicate='intersects')

#         gdf_n_to_overlay=gdf_n.loc[gdf_n.index.isin(joined.index)]
        
#         gdf_final=gpd.overlay(
#         gdf_n1,       # premier jeu de polygones
#         gdf_n_to_overlay,   # second jeu de polygones
#         how='union', keep_geom_type=False 
#         )
#         gdf_final = gdf_final[gdf_final.geometry.apply(lambda g: isinstance(g, (Polygon, MultiPolygon)))]
#         gdf_final['year'] = y_n1
#         gdf_final = gdf_final[['year', 'geometry']]

#         # 8) On ajoute le GeoDataFrame N+1 traité à la liste
#         gdf_homogenized_list.append(gdf_final)

#     # 9) Concaténation de toutes les années traitées
#     gdf_homogenized = pd.concat(gdf_homogenized_list, ignore_index=True)

#     # 10) Tri final par année et reset index
#     return (
#         gdf_homogenized
#           .sort_values(by='year', ascending=True)
#           .reset_index(drop=True)
#     )

-----------------------------------------

-----------------------------------------

 importing polygonized shp

In [26]:
gdf_glacier=gpd.read_file("C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1/Shapefile/gdf_glacier.shp")

In [27]:
gdf_water=gpd.read_file("C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1/Shapefile/gdf_water.shp")

## Adding year to rgi

In [28]:
geom_RGI_FR_2015_to_export_year = add_year_column(geom_RGI_FR_2015_to_export, 2015)
geom_RGI_FR_2000_to_export_year = add_year_column(geom_RGI_FR_2000_to_export, 2000)

# I. FILTERING GLACIER

## filtering glacier outside RGI

In [29]:
glacier_inside_rgi_2000=exclude_outside_RGI_glacier(gdf_glacier, geom_RGI_FR_2000_to_export_year,2000)

In [30]:
glacier_inside_rgi_2015=exclude_outside_RGI_glacier(gdf_glacier, geom_RGI_FR_2015_to_export_year,2015)

## Merging and conserving the filtered years

In [31]:
glacier_filtered_RGI_merged = []

# Ajout des années 1984 à 2000 depuis glacier_insidee_rgi_2000
for year in range(1984, 2001):
    glacier_filtered_RGI_merged.append(glacier_inside_rgi_2000[glacier_inside_rgi_2000["year"]==year])

# Ajout des années 2001 à 2015 depuis glacier_inside_rgi_2015
for year in range(2001, 2016):
    glacier_filtered_RGI_merged.append(glacier_inside_rgi_2015[glacier_inside_rgi_2015["year"]==year])

# Ajout des années 2016 à 2024 depuis glacier_inside
for year in range(2016, 2025):
    glacier_filtered_RGI_merged.append(gdf_glacier[gdf_glacier["year"]==year])

In [32]:
glacier_filtered_RGI_merged_gpd=pd.concat(glacier_filtered_RGI_merged, ignore_index=True)

## Time filtering (5 years)

In [33]:
time_filtered_glacier_decroissant=glacier_time_filtering_propre(glacier_filtered_RGI_merged_gpd)

In [34]:
time_filtered_glacier_propre_total=glacier_time_filtering_propre_total(glacier_filtered_RGI_merged_gpd)

In [35]:
# m=time_filtered_glacier_decroissant.explore(color='red',name='glacier')
# time_filtered_glacier_test.explore(color='blue',name='filter',m=m)
# folium.LayerControl().add_to(m)
# m

In [36]:
# time_filtered_glacier_decroissant.to_file("C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/gdf_glacier_filtrage_5ans.shp")

In [37]:
# time_filtered_glacier_propre.to_file("C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/gdf_glacier_filtrage_total.shp")

-----------------------------------------

# II. FILTERING WATER

### filtering water error in glacier

In [38]:
water_in_glacier, water_outside=filter_water_by_glacier_gpd(gdf_water, gdf_glacier)

### filtering water in RGI

In [39]:
water_outside_rgi_2000 = exclude_inside_RGI_water(water_outside, geom_RGI_FR_2000_to_export_year,2000)

In [40]:
water_outside_rgi_2015 = exclude_inside_RGI_water(water_outside, geom_RGI_FR_2015_to_export_year,2015)

Merging and conserving the filtered years

In [41]:
water_filtered_RGI_merged = []

# Ajout des années 1984 à 2000 depuis water_outside_rgi_2000
for year in range(1984, 2001):
    water_filtered_RGI_merged.append(water_outside_rgi_2000[water_outside_rgi_2000["year"]==year])

# Ajout des années 2001 à 2015 depuis water_outside_rgi_2015
for year in range(2001, 2016):
    water_filtered_RGI_merged.append(water_outside_rgi_2015[water_outside_rgi_2015["year"]==year])

# Ajout des années 2016 à 2024 depuis water_outside
for year in range(2016, 2025):
    water_filtered_RGI_merged.append(water_outside[water_outside["year"]==year])

In [42]:
water_filtered_RGI_merged_gpd=pd.concat(water_filtered_RGI_merged, ignore_index=True)

## Clipping water to  lakes classif (no buffer, see validation)

In [53]:
# 1. Reprojeter en Lambert-93 (mètres)
lake_shape_projected = lake_classif_inrae_clean_gt900m2.to_crs(epsg=4326)

# # Filtrer les petits lacs (< 900 m²)
# lake_shape_projected["area_m2"] = lake_shape_projected.geometry.area
# lake_shape_filtered = lake_shape_projected[lake_shape_projected["area_m2"] >= 900].copy()

# # 2. Appliquer le buffer de 15 m (1/2 pixel landsat)
# lake_shape_buffered = lake_shape_filtered.copy()
# lake_shape_buffered["geometry"] = lake_shape_filtered.geometry

# # 3. Reprojeter en EPSG:4326 
# lake_shape_buffered = lake_shape_buffered.to_crs(epsg=4326)

clipping water classification to lakes shape

In [54]:
gdf_lake_unfiltered=gpd.clip(water_filtered_RGI_merged_gpd, lake_shape_projected)

## Time filtering (5 years)

In [55]:
time_filtered_water_5y=water_time_filtering_propre(gdf_lake_unfiltered)

In [56]:
time_filtered_water_propre=water_time_filtering_propre_total(gdf_lake_unfiltered)

In [57]:
#time_filtered_water_propre.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/gdf_test.shp')

In [58]:
lakes_not_clipped=water_time_filtering_propre_total(water_filtered_RGI_merged_gpd)

In [61]:
len(time_filtered_water_propre[time_filtered_water_propre['year']==2024])

102

In [62]:
# lakes_not_clipped.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_unfiltered/gdf_lake_not_clipped.shp')

In [63]:
# m=gdf_lake_unfiltered.explore(color='red',name='water')
# time_filtered_water_5y.explore(color='blue',name='filter',m=m)
# folium.LayerControl().add_to(m)
# m

## Excluding water in glacier 2024

In [64]:
gdf_lake_unfiltered_def_2024=exclude_water_inside_geometry(gdf_lake_unfiltered,time_filtered_glacier_propre_total[time_filtered_glacier_propre_total['year'] == 2024])

In [65]:
gdf_lake_filtered_5y_def_2024=exclude_water_inside_geometry(time_filtered_water_5y,time_filtered_glacier_propre_total[time_filtered_glacier_propre_total['year'] == 2024])

In [66]:
gdf_lake_filtered_total_def_2024=exclude_water_inside_geometry(time_filtered_water_propre,time_filtered_glacier_propre_total[time_filtered_glacier_propre_total['year'] == 2024])

In [69]:
len(gdf_lake_filtered_total_def_2024[gdf_lake_filtered_total_def_2024['year']==2024])

102

In [68]:
len(gdf_lake_unfiltered_def_2024[gdf_lake_unfiltered_def_2024['year']==2000])

64

In [71]:
gdf_lake_unfiltered_def_2024.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_unfiltered/gdf_lake_sans_filtrage.shp')

In [55]:
# gdf_lake_filtered_5y_def_2024.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/gdf_lake_filtrage_5ans.shp')

In [56]:
# gdf_lake_filtered_total_def_2024.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/gdf_lake_filtrage_total.shp')

---------------------------------

# III. Applying Conditions


In [57]:
# import geopandas as gpd
# import pandas as pd


# def homogenize_lakes_total(gdf):
#     years = sorted(gdf['year'].unique())

#     # 1) On démarre la liste avec la toute première année 
#     gdf_homogenized_list = [gdf[gdf['year'] == years[0]]]

#     # 2) On boucle sur toutes les années 
#     for year in years:
#         y_n  = year
#         y_n1 = year + 1

#         # 3) On prend les données brutes de chaque année
#         gdf_n  = gdf[gdf['year'] == y_n].copy()
#         gdf_n1 = gdf[gdf['year'] == y_n1].copy()

#         # 4) Préparer index pour le suivi dans gdf_n1
#         gdf_n1 = gdf_n1.reset_index(drop=True)
#         gdf_n1['index_n1'] = gdf_n1.index

#         # 5) Jointure spatiale : qui de N intersecte N+1 ?
#         joined = gdf_n.sjoin(gdf_n1, how='inner', predicate='intersects')

#         gdf_n_to_overlay=gdf_n.loc[gdf_n.index.isin(joined.index)]
        
#         gdf_final=gpd.overlay(
#         gdf_n1,       # premier jeu de polygones
#         gdf_n_to_overlay,   # second jeu de polygones
#         how='union', keep_geom_type=False 
#         )
#         gdf_final = gdf_final[gdf_final.geometry.apply(lambda g: isinstance(g, (Polygon, MultiPolygon)))]
#         gdf_final['year'] = y_n1
#         gdf_final = gdf_final[['year', 'geometry']]

#         # 8) On ajoute le GeoDataFrame N+1 traité à la liste
#         gdf_homogenized_list.append(gdf_final)

#     # 9) Concaténation de toutes les années traitées
#     gdf_homogenized = pd.concat(gdf_homogenized_list, ignore_index=True)

#     # 10) Tri final par année et reset index
#     return (
#         gdf_homogenized
#           .sort_values(by='year', ascending=True)
#           .reset_index(drop=True)
#     )

water(n)-----> water(n+1)

In [58]:
# gdf_homog_2016 = homogenize_lakes_2015_2016(gdf_lake_filtered_total_def_2024)

In [59]:
# gdf_lake_homogenized=homogenize_lakes_total(gdf_lake_filtered_total_def_2024)

In [60]:
# from shapely.geometry import Polygon, MultiPolygon

# gdf_polygons_only = gdf_lake_homogenized[gdf_lake_homogenized.geometry.apply(lambda geom: isinstance(geom, (Polygon, MultiPolygon)))]
# gdf_polygons_only.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/gdf_lake_homogenized.shp')


lake classification > glacier

In [72]:
lake_diss=gdf_lake_filtered_total_def_2024.loc[gdf_lake_filtered_total_def_2024['year']>1984].dissolve(by='year')

In [73]:
glacier_diss=time_filtered_glacier_propre_total.dissolve(by='year')

In [74]:
glacier_classified=glacier_diss.assign(geom_diff=glacier_diss.difference(lake_diss))

In [75]:
glacier_final=glacier_classified.drop(columns=["geometry"]).rename(columns={"geom_diff":"geometry"}).set_geometry("geometry").explode()

In [76]:
glacier_final=glacier_final.reset_index(drop=False)

# merging and exporting the gdf

In [77]:
# Ajouter une colonne source à chacun
gdf_lake_def =gdf_lake_filtered_total_def_2024.copy()
gdf_lake_def['source'] = 'water'

gdf_glacier_def = glacier_final.copy()
gdf_glacier_def['source'] = 'glacier'

# Fusionner par année sur la plage souhaitée
water_and_glacier_gdf = gpd.GeoDataFrame(
    pd.concat(
        [
            pd.concat([
                gdf_lake_def[gdf_lake_def['year'] == year],
                gdf_glacier_def[gdf_glacier_def['year'] == year]
            ])
            for year in range(1985, 2025)
        ],
        ignore_index=True
    ),
    crs=gdf_lake_filtered_total_def_2024.crs
)


In [78]:
water_and_glacier_gdf['value'] = water_and_glacier_gdf['source'].map({'water': 2, 'glacier': 1})

In [79]:
water_and_glacier_gdf.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/glacier_and_lake_final.shp')

In [83]:
# Séparer les lacs (eau)
lake_def_clean = water_and_glacier_gdf[water_and_glacier_gdf["source"] == "water"].copy()

# Séparer les glaciers
glacier_def_clean = water_and_glacier_gdf[water_and_glacier_gdf["source"] == "glacier"].copy()

In [ ]:
glacier_def_clean

Final Export

In [84]:
lake_def_clean.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/lake_def_clean.shp')

In [85]:
glacier_def_clean.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/glacier_def_clean.shp')

## Associating each lake/glacier with the same id

In [86]:
lake_def_clean_to_id=gpd.read_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/lake_def_clean.shp')
glacier_def_clean_to_id=gpd.read_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/glacier_def_clean.shp')

In [87]:
lake_def_clean_to_id = lake_def_clean_to_id.to_crs(lake_classif_inrae_clean_gt900m2.crs)

Glacier_id d'après le lia_id

In [88]:
gdf_lia_vdef

,index,objectid,nom,latitude,longitude,methodo,massif,min_,max_,surface,perimetre,expo_moy,expo,Ref supp,WGI,GLIMS_ID,geometry
0,0,9004.0,Glaciers de Chauvet,"44°32′57,875″N","6°50′39,030″E",None,Ubaye,NaN,NaN,5.885395e+05,5772.364977,None,NaN,None,F4N01094A01,G006851E44548N,"POLYGON ((1004433.322 6390961.318, 1004398.008..."
1,1,9005.0,glacier oriental de Marinet,"44°33′02,227″N","6°52′10,609″E",None,Ubaye,NaN,NaN,3.207437e+05,3166.495587,None,NaN,None,F4N01094A02,G006862E44550N,"POLYGON ((1007198.229 6390761.915, 1007191.196..."
2,2,8839.0,Glacier de Faravel,"44°42′36,319″N","6°24′28,152″E",geomorphologie,Ecrins,2566.0,2997.0,3.538555e+05,2763.978741,N,13.927592,None,F4N01082A01,G006406E44709N,"POLYGON ((969598.839 6407239.658, 969626.585 6..."
3,3,8695.0,Glacier de l'Aup,"44°44′47,630″N","6°11′51,801″E",geomorphologie,Ecrins,1741.0,3035.0,1.029900e+06,6666.719017,N,338.580597,None,F4N01172H04,G006197E44744N,"POLYGON ((952378.945 6410296.338, 952375.507 6..."
4,4,8965.0,Glacier deSubeyrar du Loup,"44°48′42,122″N","6°20′52,075″E",geomorphologie,Ecrins,2025.0,3418.0,1.015384e+06,5609.235898,SE,138.814476,None,F4N01083A09,G006348E44814N,"POLYGON ((964652.341 6418724.453, 964678.255 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,902,8854.0,Glacier du Vallon Cros,"44°42′18,498″N","6°29′22,038″E",None,Ecrins,NaN,NaN,6.396253e+04,1169.035836,None,NaN,None,None,None,"POLYGON ((976148.108 6406629.297, 976119.748 6..."
903,903,8842.0,Glacier de la Montagne d'Aout,"44°42′12,097″N","6°28′33,926″E",None,Ecrins,NaN,NaN,1.618797e+05,2025.513795,None,NaN,None,None,None,"POLYGON ((975206.64 6406314.865, 975176.132 64..."
904,904,8781.0,Glacier du Col du Chéret,"44°53′07,184″N","6°18′50,884″E",geomorphologie + M. Le Roy,Ecrins,2786.0,3074.0,1.108246e+05,1585.908674,E,73.829815,None,None,None,"POLYGON ((961504.525 6425988.63, 961491.059 64..."
905,905,8638.0,Glacier du Grand Armet Ouest,"44°58′45,653″N","5°55′41,338″E",None,Taillefer,NaN,NaN,4.846356e+04,1589.931916,None,NaN,None,None,None,"POLYGON ((930839.053 6435187.55, 930763.856 64..."


In [89]:
glacier_def_clean_to_id=glacier_def_clean_to_id.to_crs(gdf_lia_vdef.crs)

In [90]:
join_glac = glacier_def_clean_to_id.sjoin(gdf_lia_vdef, how='inner', predicate='intersects')

# 3. Créer une colonne 'glacier_lia_id' dans glacier_final_1985 avec l'ID de gdf_lia
# join_glac contient à la fois les index de glacier_final_1985 (index_left) et de gdf_lia (index_right), et la colonne 'ID' de gdf_lia
glacier_def_clean_ided=glacier_def_clean_to_id.copy()
glacier_def_clean_ided.loc[join_glac.index, 'glacier_lia_id'] = join_glac['index'].values


In [92]:
glacier_def_clean_ided=glacier_def_clean_ided

In [94]:
glacier_def_clean_ided.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Glacier/glacier_def_clean_with_id.shp')

C:\Users\ronir\AppData\Local\Temp\ipykernel_12576\2255593895.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  glacier_def_clean_ided.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Glacier/glacier_def_clean_with_id.shp')
C:\Users\ronir\anaconda3\envs\gee\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'glacier_lia_id' to 'glacier_li'
  ogr_write(


Pour les lacs, utilisation de la classif INRAE

In [95]:
join_lake=lake_def_clean_to_id.sjoin(lake_classif_inrae_clean_gt900m2, how='inner', predicate='intersects')

In [96]:
lake_def_clean_ided=lake_def_clean_to_id.copy()
lake_def_clean_ided.loc[join_lake.index, 'lake_id'] = join_lake['id'].values

In [133]:
best_matches

,year,source,value,id,Remarques,Evolution,Date,Surface,Categorie,Barrage,NAME,ID_2_2,INSEE,NOM,ID_1,ID_3,area_m2,geometry,intersection_area,index_left
2066,2024,water,2,377,GL,V,2021,113626,P,None,None,NaN,None,None,377.0,NaN,113473.964177,"POLYGON ((946026.198 6455261.835, 946047.359 6...",93664.170148,2066
1963,2023,water,2,377,GL,V,2021,113626,P,None,None,NaN,None,None,377.0,NaN,113473.964177,"POLYGON ((946026.198 6455261.835, 946047.359 6...",92041.683382,1963
1864,2022,water,2,377,GL,V,2021,113626,P,None,None,NaN,None,None,377.0,NaN,113473.964177,"POLYGON ((946026.198 6455261.835, 946047.359 6...",89226.988219,1864
516,2002,water,2,367,GL,SG,2022,95013,P,None,None,NaN,None,None,367.0,NaN,94886.780436,"POLYGON ((948900.393 6454477.333, 948911.2 645...",88399.389059,516
1517,2018,water,2,367,GL,SG,2022,95013,P,None,None,NaN,None,None,367.0,NaN,94886.780436,"POLYGON ((948900.393 6454477.333, 948911.2 645...",88160.037272,1517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,2022,water,2,245,GL,V,2022,106785,P,None,None,NaN,None,None,245.0,NaN,106622.284269,"POLYGON ((994414.376 6485626.428, 994422.868 6...",18.961697,1886
1206,2014,water,2,207,GL,V,2021,81115,P,None,None,NaN,None,None,207.0,NaN,81027.918376,"POLYGON ((955287.029 6423163.282, 955287.134 6...",7.958795,1206
1915,2023,water,2,200,None,SG,2022,77915,P,None,None,NaN,None,None,200.0,NaN,77812.185998,"POLYGON ((1004785.183 6456465.342, 1004786.501...",0.895263,1915
2013,2024,water,2,200,None,SG,2022,77915,P,None,None,NaN,None,None,200.0,NaN,77812.185998,"POLYGON ((1004785.183 6456465.342, 1004786.501...",0.895263,2013


In [129]:
len(best_matches[best_matches['year']==2022].id.unique())

79

In [102]:
len(lake_def_clean_ided[lake_def_clean_ided['year']==2024]['lake_id'])

102

In [120]:
join = lake_def_clean_ided[lake_def_clean_ided['year']==2022].sjoin(lake_classif_inrae_clean_gt900m2, how='inner', predicate = 'intersects')

In [121]:
test = lake_def_clean_ided.loc[lake_def_clean_ided.index.isin(join.index)]

In [122]:
len(test['lake_id'].unique())

78

In [110]:
lake_def_clean_ided[lake_def_clean_ided['year']==2024].explore()

In [111]:
lake_def_clean_ided.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Lake/lake_def_clean_with_id.shp')

same with unfiltered_lake

In [112]:
lake_def_clean_not_filtered_to_id=gpd.read_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_unfiltered/gdf_lake_sans_filtrage.shp')

In [113]:
lake_def_clean_not_filtered_to_id=lake_def_clean_not_filtered_to_id.sort_values(by='year', ignore_index=True)

In [114]:
# lake_def_clean_not_filtered_to_id=lake_def_clean_not_filtered_to_id.assign(lake_id='')

In [115]:
join_lake_unfilt=lake_def_clean_not_filtered_to_id.sjoin(lake_shape_clipped, how='inner', predicate='intersects')

In [116]:
join_lake_unfilt

,year,geometry,index_right,id,Remarques,Evolution,Date,Surface,Categorie,Barrage,NAME,ID_2_2,INSEE,NOM,ID_1,ID_3
0,1984,"POLYGON ((6.24635 44.87732, 6.24679 44.87732, ...",34,30,GL,SG,2021,76019,P,None,None,NaN,None,None,30.0,NaN
1,1984,"POLYGON ((7.11008 45.27779, 7.11008 45.27725, ...",303,198,GL,SG,2022,8034,P,None,None,NaN,None,None,198.0,NaN
2,1984,"POLYGON ((6.84634 45.13522, 6.84635 45.13521, ...",342,1611,None,NL,2022,1941,P,None,None,NaN,None,None,NaN,1611.0
3,1984,"POLYGON ((6.91092 45.41873, 6.91092 45.41846, ...",389,249,None,SG,2022,20737,None,None,None,NaN,None,None,249.0,NaN
4,1984,"POLYGON ((6.11995 45.14061, 6.11995 45.14059, ...",100,37,GL,SG,2021,89556,P,None,None,NaN,None,None,37.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861,2024,"POLYGON ((7.11018 45.26377, 7.11021 45.26376, ...",283,196,GL,V,2022,13967,P,None,None,196.0,73040,Bessans,196.0,NaN
2861,2024,"POLYGON ((7.11018 45.26377, 7.11021 45.26376, ...",282,195,GL,V,2022,13967,P,None,None,195.0,73040,Bessans,195.0,NaN
2862,2024,"POLYGON ((6.97257 45.5842, 6.97256 45.58415, 6...",662,366,None,SG,2022,3196,P,None,None,NaN,None,None,366.0,NaN
2863,2024,"POLYGON ((7.06628 45.21365, 7.06642 45.21353, ...",275,1482,GL,NL,2022,4728,P,None,None,NaN,None,None,NaN,1482.0


In [117]:
lake_def_clean_not_filtered_ided=lake_def_clean_not_filtered_to_id.copy()
lake_def_clean_not_filtered_ided.loc[join_lake_unfilt.index, 'lake_id'] =join_lake_unfilt['id'].values

In [118]:
lake_def_clean_not_filtered_ided

,year,geometry,lake_id
0,1984,"POLYGON ((6.24635 44.87732, 6.24679 44.87732, ...",30.0
1,1984,"POLYGON ((7.11008 45.27779, 7.11008 45.27725, ...",198.0
2,1984,"POLYGON ((6.84634 45.13522, 6.84635 45.13521, ...",1611.0
3,1984,"POLYGON ((6.91092 45.41873, 6.91092 45.41846, ...",249.0
4,1984,"POLYGON ((6.11995 45.14061, 6.11995 45.14059, ...",37.0
...,...,...,...
2860,2024,"POLYGON ((6.9142 45.39798, 6.91425 45.39792, 6...",248.0
2861,2024,"POLYGON ((7.11018 45.26377, 7.11021 45.26376, ...",195.0
2862,2024,"POLYGON ((6.97257 45.5842, 6.97256 45.58415, 6...",366.0
2863,2024,"POLYGON ((7.06628 45.21365, 7.06642 45.21353, ...",1482.0


In [119]:
lake_def_clean_not_filtered_ided.to_file('C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/lake_def_clean_not_filtered_with_id.shp')

---------------------------------

---------------------------------